<a href="https://colab.research.google.com/github/koh-ts/human_activity_recognition/blob/master/human_activity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn import svm
import pandas_profiling as pdp
from IPython.display import HTML
import lightgbm as lgb

Mount your Google Drive. In this notebook, we assume that 'report2' folder is placed directly under 'My Drive'.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Load the data.

In [0]:
root_dir="/content/drive/My Drive/report2/"
nsample = 2000

X_train = np.array(pd.read_csv(root_dir+"X_train.csv",header=None))
y_train = np.array(pd.read_csv(root_dir+"y_train.csv",header=None).ix[:,0])
X_test = np.array(pd.read_csv(root_dir+"X_test.csv",header=None))

In [0]:
XTrain = X_train[:nsample,:] #use the first 3000 samples for training
yTrain = y_train[:nsample]
XVal = X_train[nsample:,:] #use the rests for validation
yVal = y_train[nsample:]
alldata = pd.concat([XTrain1, X_test1], ignore_index=True)

In [0]:
XTrain1.head(3)

In [0]:
# profile1 = pdp.ProfileReport(alldata)
# profile1.to_file(outputfile="myoutputfile1.html")
# HTML(filename="myoutputfile1.html")

Train a linear SVM classifier

In [0]:
print("Training linear SVM classifier.")
clf = svm.LinearSVC()
clf.fit(XTrain,yTrain)

Evaluate training end validation scores.

In [0]:
yHatTrain = clf.predict(XTrain)
yHatVal = clf.predict(XVal)
print("Training score ", len((np.where(yHatTrain == yTrain))[0])*1.0/XTrain.shape[0])
print("Validation score ", len((np.where(yHatVal == yVal))[0])*1.0/XVal.shape[0])

In [0]:
train_data_set = lgb.Dataset(XTrain, yTrain)
valid_data_set = lgb.Dataset(XVal, yVal, reference=train_data_set)

params = {'boosting_type': 'gbdt',
          'objective': 'multiclass',
          'num_class': 7,
          'metric': {'multi_error'},
          'num_leaves': 32,
          'learning_rate': 0.05,
          'feature_fraction': 0.9, #0.9
          'bagging_fraction': 0.8, #0.8
          'bagging_freq': 7}

In [0]:
gbm = lgb.train(params, train_data_set, num_boost_round=300, valid_sets=[train_data_set, valid_data_set], early_stopping_rounds=100)

Predict test data and save the result.

In [0]:
yHatTest_gbm_raw = gbm.predict(X_test)
yHatTest_gbm = np.argmax(yHatTest_gbm_raw, axis=1)
np.savetxt(root_dir+'result_gbm.txt', yHatTest_gbm)

In [0]:
yHatTest = clf.predict(X_test)
np.savetxt(root_dir+'result_svm.txt', yHatTest)